# OpenStreetMap Data Wrangling

## Project Overview

Choose any area of the world on https://www.openstreetmap.org and use data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for a part of the world that you care about. Choose to learn SQL or MongoDB and apply your chosen schema to the project.

### Chosen Area

Raleigh, North Carolina, USA:

- https://www.openstreetmap.org/relation/179052
- https://mapzen.com/data/metro-extracts/metro/raleigh_north-carolina/

A 9 MB unprocessed map sample can be downloaded [here](../data/raleigh_north-carolina_sample.zip).

I picked this region since I'd spent almost three months there on my last business trip to the USA, more specifically to the Research Triangle. I visited quite a lot of places, but I realize there's much more I didn't get to see, so wrangling and then querying this data is a perfect opportunity to learn more about the city, if only virtually this time around. Might be I'll even have a chance to contribute to the OpenStreetMap community, who knows.

## Exploratory Code

This section contains the Python code I used to investigate the dataset before cleaning and uploading it to an sqlite3 database.

In [1]:
# importing the necessary libraries
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

In [5]:
# function that iteratively parses the map file and counts the number of top-level tags
def count_tags(filename):
        tags = {}
        for event, elem in ET.iterparse(filename):
            t = elem.tag
            if t in tags:
                tags[t] += 1
            else:
                tags[t] = 1
        return tags

In [3]:
# let's get a bird's eye view of the data we're dealing with
tags = count_tags('raleigh_north-carolina.osm')
pprint.pprint(tags)

{'bounds': 1,
 'member': 8535,
 'nd': 2600325,
 'node': 2307063,
 'osm': 1,
 'relation': 874,
 'tag': 872208,
 'way': 235289}


In [7]:
# patterns that will help us match various <tag> 'k' values
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

problematic_tags = []
other_tags = []

# function that assigns a specific <tag> 'k' value to one of the four categories and keeps two lists of problematic and
# unconventional 'k' values
def key_type(element, keys):
    global problematic_tags, other_tags
    if element.tag == 'tag':
        k = element.attrib['k']
        if lower.search(k):
            keys['lower'] += 1
        elif lower_colon.search(k):
            keys['lower_colon'] += 1
        elif problemchars.search(k):
            problematic_tags.append(k)
            keys['problemchars'] += 1
        else:
            other_tags.append(k)
            keys['other'] += 1
    return keys

# function that iteratively applies the key_type function to the whole map file
def process_map(filename):
    keys = {'lower': 0, 'lower_colon': 0, 'problemchars': 0, 'other': 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)
    return keys

In [8]:
# let's see how the 'k' values are distributed among the four groups
keys = process_map('raleigh_north-carolina.osm')
pprint.pprint(keys)

{'lower': 543051, 'lower_colon': 285435, 'other': 43720, 'problemchars': 2}


Most values seem to be alright, but two of them contain some problematic characters - it's worth investigating what those are.

In [10]:
print problematic_tags

['service area', 'chair lift']


Looks like those two both have a whitespace that invalidates them.

In [15]:
# what about other_tags 'k' values?
# we'll be looking at every 1000th item since there're over 40K of them
for i, o in enumerate(other_tags):
    if i % 1000 == 0: print o

gnis:Class
Street_1
tiger:zip_right_1
name_1
tiger:name_type_1
nhd-shp:fcode
nhd-shp:fdate
nhd-shp:fcode
nhd-shp:fdate
nhd-shp:fcode
nhd-shp:fdate
nhd-shp:fcode
nhd-shp:fcode
nhd-shp:fdate
nhd-shp:com_id
NHD:FCode
NHD:RESOLUTION
NHD:FCode
NHD:ComID
NHD:FTYPE
NHD:ComID
NHD:FCode
NHD:RESOLUTION
NHD:ReachCode
NHD:RESOLUTION
NHD:way_id
NHD:ComID
NHD:FCode
NHD:way_id
NHD:FCode
NHD:FCode
NHD:FCode
NHD:RESOLUTION
NHD:ComID
NHD:FDate
NHD:ReachCode
NHD:FDate
NHD:GNIS_ID
tiger:name_base_1
tiger:zip_right_1
nhd-shp:fdate
nhd-shp:fcode
tiger:name_base_1
NHD:ReachCode


This data seemingly comes from at least three different sources: Tiger GPS, National Hydrography Dataset (NHD), and Geographic Names Information System (GNIS) - judging by the prefixes.

In [6]:
OSMFILE = 'raleigh_north-carolina.osm'
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) # grabs the street type from a given string
expected = ['Street', 'Avenue', 'Boulevard', 'Drive', 'Court', 'Place',
            'Square', 'Lane', 'Road', 'Trail', 'Parkway', 'Commons']

In [4]:
# function that checks if a given street type conforms to one of the expected values
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

# function that checks if an element contains a street name
def is_street_name(elem):
    return (elem.attrib['k'] == 'addr:street')

# function that checks all the street types in a given OSM file and keeps track of all those not in line with the expected values
def audit(osmfile):
    osm_file = open(osmfile, 'r')
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=('start',)):
        if elem.tag == 'node' or elem.tag == 'way':
            for tag in elem.iter('tag'):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

In [5]:
# running this will display all unique street types, along with street names, that don't match anything we have in
# the expected list
st_types = audit(OSMFILE)
pprint.pprint(dict(st_types))

{'100': set(['100', 'Westgate Park Dr #100']),
 '1000': set(['Six Forks Road #1000']),
 '102': set(['NE Maynard Rd #102', 'Page Rd #102']),
 '17': set(['US Highway 17']),
 '206': set(['Barrett Dr Suite 206']),
 '27609': set(['27609']),
 '27609-5360': set(['27609-5360']),
 '315': set(['Kildaire Farm Road #315']),
 '501': set(['US 15;US 501']),
 '54': set(['Highway 54',
            'State Highway 54',
            'West Highway 54',
            'West NC Highway 54',
            'West State Highway 54']),
 '55': set(['Highway 55', 'NC Highway 55', 'US 55']),
 '70': set(['US 70']),
 '751': set(['NC 751', 'NC Highway 751']),
 'Alley': set(['Yates Motor Company Alley']),
 'Ave': set(['Atlantic Ave',
             'E. Winmore Ave',
             'East Winmore Ave',
             'Glenwood Ave',
             'Mountford Ave',
             'S Boylan Ave',
             'S. Boylan Ave']),
 'Blvd': set(['Airport Blvd',
              'Capital Blvd',
              'Fordham Blvd',
              'Lake Grov

Naturally enough, given a large number of contributors to the OSM project, there's a certain variation in the spelling of some of the most common street types: street (ST, St, St.), road (Rd, Rd.), parkway (Pkwy, Pky), drive (Dr, Dr.) to name a few.

## Problems Identified

While I was playing with the map data, I encountered two major problems that I had to deal with:

1. I've already mentioned this issue above: some street types have several abbreviated versions or simply contain typos, which breeds inconsistency.
2. A lot of data in the map doesn't seem to describe Raleigh itself, but rather surrounding areas.

I'm going to elaborate on both these items below.

### Inconsistent Street Type Names

If you run the *audit* function provided above, you'll see records like 'Capital Blvd', 'Southpoint Auto Park Blvd.', 'Meadowmont Village CIrcle', or 'W Rosemary St.'. For the sake of consistency, I decided to spell out each name in full, so that, for example, 'W Rosemary St.' becomes 'West Rosemary Street'. However, I realized I needed to be extra careful with cases like 'Ste N' (for it not to be turned into 'Suite North') - hence an additional check in the if statement in the *update_name* function below.

So, I created a mapping from the "incorrect" spelling to the correct one and then fixed it on the fly while iterating over the whole file.

In [2]:
mapping = {'ST': 'Street', 'St': 'Street', 'St.': 'Street', 'St,': 'Street', 'Rd.': 'Road', 'Rd': 'Road', 'Pl': 'Place',
          'PI' : 'Place', 'Pky': 'Parkway', 'Pkwy': 'Parkway', 'Ln': 'Lane', 'Dr': 'Drive', 'Dr.': 'Drive', 'Driver': 'Drive',
          'Ct': 'Court', 'Cir': 'Circle', 'CIrcle': 'Circle', 'Blvd.': 'Boulevard', 'Blvd': 'Boulevard', 'Ave': 'Avenue',
          'Ste': 'Suite', 'Ext': 'Extension', 'W': 'West', 'E': 'East', 'N': 'North', 'S': 'South'}

In [3]:
# this function corrects the street type spelling while the data is being prepared to be put into CSV files
def update_name(name, mapping):
    name = name.split()
    for n in xrange(len(name)):
        if name[n] in mapping and name[n - 1] not in ('Ste', 'Ste.', 'Suite'):
            name[n] = mapping[name[n]]
    return ' '.join(name)

### "Misplaced" Zip Codes

Before I started working on this project, I did some reading on Raleigh and found out all the zip codes in the city start with 276. However, in the course of querying the database that contained the processed data I noticed some zip codes don't match this pattern, so I wrote the following query to see what the top 20 (purely arbitrary number) zip codes are:

```SQL
SELECT tags.value, COUNT(*) cnt 
FROM (SELECT * FROM ways_tags 
      UNION 
      SELECT * FROM nodes_tags) tags
WHERE tags.key = 'postcode'
GROUP BY tags.value
ORDER BY cnt DESC
LIMIT 20;
```

It selects records from two tables (*ways_tags* and *nodes_tags*) and groups them by postal code.

The results of this query, in descending order, are below:


zip code|count
--------|---
27560|1567
27519|899
27701|685
27609|674
27705|518
27615|388
27510|328
27514|179
27513|158
27511|113
27606|106
27707|99
27516|96
27601|94
27517|74
27703|68
27704|62
27612|59
27713|51
27617|45

Out of the 20 zip codes, less than a third (6, to be precise) start with 276 and thus belong to the city of Raleigh, which means the map file contains large areas outside the city limits. Let's now see what those areas are with the help of the following aggregation:

```SQL
SELECT tags.value, COUNT(*) cnt 
FROM (SELECT * FROM ways_tags
      UNION
      SELECT * FROM nodes_tags) tags
WHERE tags.key = 'city'
GROUP BY tags.value
ORDER BY cnt DESC
LIMIT 10; /* can be dropped, as we'll later see */
```

Like the query above, it selects records from two tables (*ways_tags* and *nodes_tags*), looks at those that have 'city' in the 'key' column, and groups these records by city name (that is the value in the 'value' column).

The top ten cities produced by this query are as follows:

1. Raleigh
2. Cary
3. Morrisville
4. Durham
5. Chapel Hill
6. Carrboro
7. Research Triangle Park
8. Wake Forest
9. chapel Hill (sic!)
10. durham (sic!)

What jumps right out at you is the mistyped city names (9 and 10), which, however, can easily be fixed with a query of the following form:

```SQL
UPDATE <table_name> SET value = <correct_city_name> WHERE value = <mistyped_name>;
```

I actually had to run it quite a few times (6 or 7) before I got the error-free results, so I didn't include the exact values, but provided a template instead. The final top-1o (which is not entirely true) list looks like this:

1. Raleigh
2. Cary
3. Morrisville
4. Durham
5. Chapel Hill
6. Carrboro
7. Research Triangle Park
8. Wake Forest
9. Apex

There are "only" 9 unique cities in the dataset, after all! So, we can omit the 'LIMIT 10' line from the query we used to obtain this list.

Now, it becomes more evident the map includes Raleigh-Durham-Chapel Hill Combined Statistical Area (CSA) but is not confined to it, encompassing even a few satellite towns like Cary and Morrisville.

On the one hand, come to think of it, it does make sense to include this data together since all these cities are very tightly tied both economically and politically. On the other hand, it's surprising and even somewhat perplexing to find so much unexpected data in a map file clearly marked as describing one city - Raleigh.

I can see two possible ways of solving this issue: either separate this map file into distinct cities (it would reduce the size of the corresponding files and make for simpler processing) - one city per file - or give the file a more descriptive name to avoid potential confusion and misunderstanding among contributors and people using these map files for their own purposes.

## Overview Statistics

This section contains high-level data stats and the corresponding SQL queries used to gather them. 

### File Sizes

File name|Size
---------|----
Raleigh.db|249 MB
raleigh_north-carolina.osm|456 MB
nodes.csv|178 MB
nodes_tags.csv|1.97 MB
ways.csv|12.9 MB
ways_nodes.csv|56.9 MB
ways_tags.csv|27.9 MB

In the **Exploratory Code** section, I used the *count_tags* function to count the number of unique tags and got this result: 2,307,063 node tags and 235,289 way tags, among others. Let's now use SQL to see if new numbers match what Python gave us.

### Number of Way Tags

The query is pretty straightforward:
    
```SQL
SELECT COUNT(*) FROM ways;
```

The result is indeed 235,289!

### Number of Node Tags

All we need to do is change the name of the table in the query above:

```SQL
SELECT COUNT(*) FROM nodes;
```

And this query yields 2,307,063.

### Number of Unique Users

We'll just select and combine unique user ids from two tables like this:

```SQL
SELECT COUNT(DISTINCT(users.uid))          
FROM (SELECT uid FROM nodes
      UNION
      SELECT uid FROM ways) users;
```

Turns out 821 people have contributed to this map so far. Wouldn't it be nice to know the top contributors? Let's do just that!

### Top Contributors

The query below selects records from the *nodes* and *ways* tables and groups them by user name, effectively producing the number of contributions each of the top 10 users made:

```SQL
SELECT users.user, COUNT(*) as cnt          
FROM (SELECT user FROM nodes
      UNION ALL
      SELECT user FROM ways) users
GROUP BY users.user
ORDER BY cnt DESC
LIMIT 10;
```

The top-10 list is as follows:

user name|contributions
---------|-------------
jumbanho|1571998
JMDeMai|178885
bdiscoe|127604
woodpeck_fixbot|114991
bigal945|103478
yotann|66862
runbananas|41623
sandhill|32512
MikeInRaleigh|30150
Clay Hobbs|22107


jumbanho is way ahead of the competition with 1.5 million contributions, which is almost 10 times more than the runner-up and definitely overshadows contributors 2-10 combined!

It's also worth exploring user activity by year to see when the community members working on this map were the most active.

### Contributions by Year

We'll just need to get the year from the 'timestamp' column and group the records from two tables by year:

```SQL
SELECT STRFTIME('%Y', users.timestamp), COUNT(*) as cnt          
FROM (SELECT timestamp FROM nodes
      UNION ALL
      SELECT timestamp FROM ways) users
GROUP BY STRFTIME('%Y', users.timestamp)
ORDER BY cnt DESC;
```

Here's what the query returns:

year|contributions
----|-------------
2010|951368
2009|587266
2015|301073
2016|173996
2013|168398
2011|147953
2012|112006
2014|96906
2008|2969
2007|417

The main takeaway from these results is that this map was created 9 years ago, gaining popularity among users year after year, with major spikes in 2009 and then 2010 (when contributors were the most active) - then the user activity slumped in 2011 and experienced a few ups and downs over the years.

Below is a chart for visual support:

![](../img/contributions.png)

## Further Exploration

This section contains several SQL queries I used to gain a deeper insight into the data.

### Popular Types of Cuisine

During my time in Raleigh, I noticed that, apart from traditional American pizza and burger places, there were lots of cafes offering Mexican and Italian cuisine. Let's now see what types of cuisine are the most popular in this area:

```SQL
SELECT tags.value, COUNT(*) cnt
FROM (SELECT * FROM nodes_tags
      UNION ALL
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'cuisine'
GROUP BY tags.value
ORDER BY cnt DESC
LIMIT 10;
```

Here are the results in descending order of popularity:

type of cuisine|number of places
---------------|----------------
burger|88
sandwich|59
mexican|53
american|52
pizza|52
coffee_shop|33
italian|32
chicken|29
chinese|29
japanese|18

Indeed, both Mexican and Italian food did make the top-10 list!

### Amenities

So, cafes and restaurants are obviously pretty popular places, but what can the data tell us about other types of amenities? Let's select records from the *nodes_tags* and *ways_tags* tables and group them by the type of amenity:

```SQL
SELECT tags.value, COUNT(*) cnt
FROM (SELECT * FROM nodes_tags
      UNION ALL
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'amenity'
GROUP BY tags.value
ORDER BY cnt DESC
LIMIT 10;
```

Top 10 results look like this:

amenity|number of entries
-------|-----------------
parking|2334
bicycle_parking|576
restaurant|558
place_of_worship|537
fast_food|285
school|228
fuel|215
bench|132
waste_basket|118
bank|116

Quite a few places of worship out there! A logical enough step further would be to check what religions are represented in the area. While the number one answer is more or less predictable, it's still interesting to see what distribution the data offers.

### Religion

```SQL
SELECT tags.value, COUNT(*) cnt
FROM (SELECT * FROM nodes_tags
      UNION ALL
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'religion'
GROUP BY tags.value
ORDER BY cnt DESC;
```

Here's what we get:

religion|number of entries
--------|-----------------
christian|492
muslim|6
jewish|5
unitarian_universalist|2
bahai|1
eckankar|1
hindu|1
sai_baba|1

Given the number of Hindus in the USA, I'm surprised to see Hinduism so severely underrepresented in the dataset.

It's very important to understand the limitations of the analysis performed in this section. The data is apparently incomplete and also includes areas outside Raleigh, which leads to certain data skewness. So, we're limited to making **tentative** conclusions about the data and shouldn't try to extrapolate what modest findings we have on hand to larger regions (like the entire city of Raleigh, the state of North Carolina, or even the whole territory of the USA).

## Conclusion

After having worked with the map data for some time now, here are my key takeaways from this project, along with some improvement suggestions:

1. The data contains lots of typos (take city names as an example), and some values (like street types) just have multiple spellings, which breeds inconsistency, increases the overall size of the dataset, and, on the whole, makes it harder to work with it. It would be extremely useful to have some automatic checks (based on the rules agreed on by the core OSM community; for example: can all caps be used in records?; what's more acceptable, 'Street' or 'St' or 'St.'? and so on) in place that would prevent problematic entries from making their way into the map data. Of course, it would take a large-scale discussion to establish these filtering rules and may be problematic due to contributors being scattered all over the US and possibly the world. To make it more feasible, the core community could appoint several trusted representatives to a committee whose primary goal would be to come up with the rules in question.
2. Despite being named 'Raleigh', this map file includes a much larger area, encompassing several neighboring cities, which did take me aback the first time I looked at the data. While the reasons behind the choice to include extra areas might be clear (say, close socio-economic ties), it's still confusing to have all these cities jumbled together. I believe it would be more convenient, both from the logical and computational standpoints, to have a separate file for each city, however small it might be. One major concern here is the time and effort involved in performing this task, so the question still remains - who's going to take it upon themselves to do the splitting? Would it be a single contributor skilled enough in scripting to do it programmatically? Or are several contributors from the corresponding areas better suited for the job?
3. To grow the community and involve new members more actively, spurring competition among the contributors sounds like a good idea to me. It could be achieved by introducing some kind of a leaderboard (could be global or by region/country/city, or both) and awarding honor badges for various achievements (like the most valuable contribution (would be chosen based on the number of likes or upvotes it receives from the community), the highest number of edits in one day/month/year and anything along these lines). On the other hand, users that systematically disregard the rules and standards set out by the community could somehow be penalized or suspended from contributing for some time. The last suggestion may seem a little harsh and could potentially put off some of the contributors, but I strongly believe it's a necessary evil if the community's primary goal is to have well-cleaned, reliable, unambiguous, and easy-to-use maps.
4. This point should be studied more closely, but I noticed that some data has prefixes that indicate it comes from sources like Tiger GPS, National Hydrography Dataset (NHD), and Geographic Names Information System (GNIS). And this data was highly likely added to the map file automatically (why would a person add all these prefixes manually in the first place?), by a scraper or a crawler created by a skilled programmer. If the community could leverage the expertise of such people, combined with the automatic rule-based filters I proposed in item 1, this map file (and not only this, but potentially all of them) could be greatly improved given a programmatically generated influx of well-cleaned data that requires minimum interference from human editors. The only issue I see here (apart from the one I highlighted in item 1 above) is that all the automation is done by humans and is thus error-prone by default, so this initiative may require quite a few skillful programmers to meticulously double-check everything and to maintain the whole data pipeline as well as make adjustments and improvements to it as new data sources appear and as data itself is evolving.